In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nir-generated-answers/generated_results_with_f1_llama2_chegeka.csv
/kaggle/input/nir-generated-answers/chegeka-shap1.xpynb
/kaggle/input/nir-generated-answers/generated_results_chegeka_mistral.csv
/kaggle/input/nir-generated-answers/rudetox-nir (1).ipynb
/kaggle/input/nir-generated-answers/chgk_generated_results_with_f1_mistral.csv
/kaggle/input/nir-generated-answers/updated_new_vikhr_generated_results_vikhr.csv
/kaggle/input/nir-generated-answers/updated_new_llama3.2_chegeka_generated_results.csv
/kaggle/input/nir-generated-answers/ruopenbookqa_vikhr_generated_results.csv
/kaggle/input/nir-generated-answers/new_lcs_llama3.2_generated_results.csv
/kaggle/input/nir-generated-answers/Llama2_rutedox_generated_results.csv
/kaggle/input/nir-generated-answers/ruopenbookqa_tinyllama_generated_results.csv
/kaggle/input/nir-generated-answers/ruopenbookqa_mistral_generated_results.csv
/kaggle/input/nir-generated-answers/new_tinyLlama_lcs_generated_results.csv
/kaggle/input/nir-gene

# Attention Matrix with Bertviz

In [6]:
!pip install bertviz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("Doctor-Shotgun/TinyLlama-1.1B-32k-Instruct")
model = AutoModelForCausalLM.from_pretrained("Doctor-Shotgun/TinyLlama-1.1B-32k-Instruct")


inputs = tokenizer("""Даны две строки в качестве входных данных:\n"CZUWAQSJOS LTHWJNYRZ"\nУкажите длину их наибольшей общей подпоследовательности. Ответ:""", return_tensors='pt')
out = model(**inputs, output_attentions=True)

In [4]:
out

CausalLMOutputWithPast(loss=None, logits=tensor([[[ -8.7125,  -7.3342,   0.9379,  ...,  -2.3946,  -5.0115,  -2.4763],
         [ -3.6088,  -3.4707,   1.8814,  ...,  -0.8597,   0.9936,  -1.1077],
         [ -4.9860,  -4.9080,   0.5335,  ...,  -2.6366,  -7.5021,  -3.3226],
         ...,
         [ -4.8510,  -4.7506,   2.4280,  ...,  -1.3304,  -1.8262,   2.3766],
         [ -6.0728,  -6.4435,   5.0763,  ...,  -1.5206,  -3.9572,  -0.1587],
         [-10.5009, -10.2521,   8.0575,  ...,  -3.3730,  -8.0551,  -2.7359]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=None, hidden_states=None, attentions=(tensor([[[[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.5213, 0.4787, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.3233, 0.3428, 0.3338,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0096, 0.0100, 0.0095,  ..., 0.0099, 0.0000, 0.0000],
          [0.0100, 0.0095, 0.0096,  ..., 0.0095, 0.0103, 0.0000],
          [0.0051, 0.0112, 0.0074,  ..., 0.

In [ ]:
from transformers import utils
from bertviz import model_view


utils.logging.set_verbosity_error()  # Suppress standard warnings

attention = out['attentions']  # Retrieve attention from model outputs
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])  # Convert input ids to token strings
model_view(attention, tokens)  # Display model view

In [ ]:
from bertviz import head_view
head_view(attention, tokens)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from bertviz import model_view
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Doctor-Shotgun/TinyLlama-1.1B-32k-Instruct")
model = AutoModelForCausalLM.from_pretrained("Doctor-Shotgun/TinyLlama-1.1B-32k-Instruct")

# Similar prompts
prompts = [
    "Как называется фильм, в котором играет Евгений Матвеев?",
    "Как называется фильм с Евгением Матвеевым в главной роли?",
]

# Prepare input tensors
inputs = [tokenizer(prompt, return_tensors='pt') for prompt in prompts]

# Get attention outputs
attention_outputs = []

for input_tensor in inputs:
    with torch.no_grad():
        output = model(**input_tensor, output_attentions=True)
        attention_outputs.append(output)

# Compare attention for each prompt
for i, output in enumerate(attention_outputs):
    attention = output.attentions
    tokens = tokenizer.convert_ids_to_tokens(inputs[i]['input_ids'][0])
    print(f"Attention for Prompt {i+1}:")
    model_view(attention, tokens)

# LIME

In [ ]:
def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', str(text)).strip()

def apply_lime_to_dataset(csv_path, model_name):
    try:
        df = pd.read_csv(csv_path)
        X = df['generated_text'].apply(clean_text)
        y = df['expected_answer']
        
        if len(y.unique()) < 2:
            print(f"Error: Not enough unique classes in the dataset for {model_name}")
            return None
        
        model = make_pipeline(TfidfVectorizer(ngram_range=(1, 2), min_df=2), LogisticRegression())
        model.fit(X, y)
        
        explainer = LimeTextExplainer(class_names=['Incorrect', 'Correct'])
        feature_importance = {}
        
        for idx in tqdm(range(len(X)), desc=f"Processing {model_name}"):
            text = X.iloc[idx]
            exp = explainer.explain_instance(text, model.predict_proba, num_features=10)
            
            for feature, importance in exp.as_list():
                if feature in feature_importance:
                    feature_importance[feature].append(abs(importance))
                else:
                    feature_importance[feature] = [abs(importance)]
        
        avg_importance = {feature: np.mean(importances) for feature, importances in feature_importance.items()}
        sorted_features = sorted(avg_importance.items(), key=lambda x: x[1], reverse=True)
        
        top_features = sorted_features[:20]
        
        print(f"\nTop 20 LIME Features for {model_name}:")
        for feature, importance in top_features:
            print(f"{feature}: {importance}")
        
        print(f"Total unique features found: {len(avg_importance)}")
        print(f"Model accuracy: {model.score(X, y):.4f}")
        
        return top_features
        
    except Exception as e:
        print(f"An error occurred while processing {model_name}: {str(e)}")
        return None

def visualize_lime_results(results):
    df_list = []
    for model, features in results.items():
        df = pd.DataFrame(features, columns=['Feature', 'Importance'])
        df['Model'] = model
        df_list.append(df)
    
    df_combined = pd.concat(df_list, ignore_index=True)
    
    top_features = df_combined.groupby('Feature')['Importance'].mean().nlargest(10).index
    
    df_plot = df_combined[df_combined['Feature'].isin(top_features)]
    
    plt.figure(figsize=(15, 10))
    sns.barplot(x='Feature', y='Importance', hue='Model', data=df_plot)
    plt.title('Top 10 LIME Features Across Models', fontsize=16)
    plt.xlabel('Features', fontsize=12)
    plt.ylabel('Importance', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Model', title_fontsize='12', fontsize='10')
    plt.tight_layout()
    plt.savefig('lime_features_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

lime_results = {}
for path, name in zip(dataset_paths, model_names):
    top_features = apply_lime_to_dataset(path, name)
    if top_features:
        lime_results[name] = top_features

visualize_lime_results(lime_results)

# SHAP analogue

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity(prompt, generated_text, vectorizer):
    """Compute cosine similarity between prompt and generated text using TF-IDF vectors."""
    prompt_vector = vectorizer.transform([prompt])
    generated_vector = vectorizer.transform([generated_text])
    return cosine_similarity(prompt_vector, generated_vector)[0, 0]

def analyze_input_impact_alternative(df, text_column='prompt', target_column='generated_text'):
    vectorizer = TfidfVectorizer(max_features=100) 
    X = vectorizer.fit_transform(df[text_column])
    
    feature_names = vectorizer.get_feature_names_out()
    original_similarities = [compute_similarity(row[text_column], row[target_column], vectorizer) for _, row in df.iterrows()]
    
    feature_importances = []
    
    for i, feature in enumerate(feature_names):
        modified_df = df.copy()
        modified_df[text_column] = modified_df[text_column].apply(lambda x: x.replace(feature, ''))
        modified_similarities = [compute_similarity(row[text_column], row[target_column], vectorizer) for _, row in modified_df.iterrows()]
        
        # Compute the impact of removing this feature
        impact = np.mean(original_similarities) - np.mean(modified_similarities)
        feature_importances.append((feature, impact))
    
    feature_importances.sort(key=lambda x: abs(x[1]), reverse=True)
    feature_importance_df = pd.DataFrame(feature_importances, columns=['feature', 'impact'])
    
    plt.figure(figsize=(12, 6))
    plt.bar(feature_importance_df['feature'][:20], feature_importance_df['impact'][:20])
    plt.title(f"Top 20 Features Impacting Similarity")
    plt.xlabel("Features")
    plt.ylabel("Impact")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()
    
    return feature_importance_df

for dataset_path in dataset_paths:
    df = pd.read_csv(dataset_path)
    if 'generated_text' not in df.columns:
        print(f"Column 'generated_text' not found in {dataset_path}. Skipping.")
        continue
    
    # Analyze feature impact
    feature_importance_df = analyze_input_impact_alternative(df, text_column='prompt', target_column='generated_text')
    output_file = f"/kaggle/working/{dataset_path.split('/')[-1].replace('.csv', '_feature_importance.png')}"
    
    plt.figure(figsize=(12, 6))
    plt.bar(feature_importance_df['feature'][:20], feature_importance_df['impact'][:20])
    plt.title(f"Top 20 Features Impacting Similarity ({dataset_path})")
    plt.xlabel("Features")
    plt.ylabel("Impact")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.show()
